In [1]:
import pandas as pd

In [5]:
# Load your data
population_df = pd.read_csv("population_state.csv")
hospital_beds_df = pd.read_csv("hospital_beds.csv")

# --- Preprocess population data ---
# Define senior age groups
senior_ages = ["60-64", "65-69", "70+", "70-74","75-79", "80-84", "85+"]  # if we want to focus on the senior citizen

pop_df = population_df[
    (population_df["sex"] == "both") &
    (population_df["age"] == "overall") &
    (population_df["ethnicity"] == "overall") 
].copy()
pop_df["population"] = (pop_df["population"] * 1000).astype(int)  # assume population is in thousands
pop_df["year"] = pd.to_datetime(pop_df["date"]).dt.year
pop_df = pop_df[["state", "year", "population"]]
# Group by state and year to sum senior population
pop_df = pop_df.groupby(["state", "year"], as_index=False)["population"].sum()


senior_pop_df = population_df[
    (population_df["sex"] == "both") &
    (population_df["age"].isin(senior_ages)) &
    (population_df["ethnicity"] == "overall") 
].copy()
senior_pop_df["population"] = (senior_pop_df["population"] * 1000).astype(int)
senior_pop_df.rename(columns={"population": "population_senior"}, inplace=True)# assume population is in thousands
senior_pop_df["year"] = pd.to_datetime(senior_pop_df["date"]).dt.year
senior_pop_df = senior_pop_df[["state", "year", "population_senior"]]
# Group by state and year to sum senior population
senior_pop_df = senior_pop_df.groupby(["state", "year"], as_index=False)["population_senior"].sum()


# --- Preprocess hospital beds data ---
beds_df = hospital_beds_df[
    (hospital_beds_df["district"] == "All Districts") &
    (hospital_beds_df["type"] == "all")
].copy()
beds_df["year"] = pd.to_datetime(beds_df["date"]).dt.year
beds_df = beds_df[["state", "year", "beds"]]

# --- Merge the datasets ---
merged_df = (
    pop_df
    .merge(beds_df, on=["state", "year"])
    .merge(senior_pop_df, on=["state", "year"])
)

# --- Calculate hospital beds per 1000 people (if focus on senior citizen only, it would be beds per 1000 senior citizen) ---
merged_df["beds_per_1000"] = round(merged_df["beds"] / (merged_df["population"] / 1000)).astype(int)
merged_df["beds_per_1000_senior"] = round(merged_df["beds"] / (merged_df["population_senior"] / 1000)).astype(int)

# View the result
print(merged_df.to_string())

merged_df.to_csv("./hospital_beds_with_population.csv", index = False)

                 state  year  population  beds  population_senior  beds_per_1000  beds_per_1000_senior
0                Johor  2015     3610300  4968             347800              1                    14
1                Johor  2016     3651800  5185             363800              1                    14
2                Johor  2017     3697000  5185             380300              1                    14
3                Johor  2018     3749400  5200             397700              1                    13
4                Johor  2019     3761200  5200             412800              1                    13
5                Johor  2020     4009700  5270             401800              1                    13
6                Johor  2021     4020000  5278             420000              1                    13
7                Johor  2022     4028300  5433             434700              1                    12
8                Kedah  2015     2096500  2653             224700        

In [2]:
#preprocess the clinic_by_state to include the district column as well
clinic_df = pd.read_csv("clinic_by_state.csv")

# Extract the district by splitting the address and taking the second last element
clinic_df['District'] = clinic_df['Full Address'].apply(lambda x: x.split(',')[-2].strip())

# Save to a new CSV or overwrite
clinic_df.to_csv("clinic_by_state_with_district.csv", index=False)

In [2]:
#creating the dataset with district level for each of the healthcare facilities
clinic_df = pd.read_csv("clinic_by_state_with_district.csv")
hospital_df = pd.read_csv("hospital_by_city_state.csv")
population_df = pd.read_csv("population_district.csv")
beds_df = pd.read_csv("hospital_beds.csv")


In [13]:
#Preprocess population data in the district:
population_df['date'] = pd.to_datetime(population_df['date'], dayfirst=True)
# latest_year = population_df['date'].dt.year.max()
latest_year = 2022
pop_filtered = population_df[
    (population_df['sex'] == 'both') &
    (population_df['age'] == 'overall') &
    (population_df['ethnicity'] == 'overall') &
    (population_df['date'].dt.year == latest_year)
].copy()

pop_filtered['population'] = pop_filtered['population'] * 1000
pop_summary = pop_filtered[['state', 'district', 'population']]

print(pop_summary.to_string())

                    state                district  population
127680              Johor              Batu Pahat    497500.0
128079              Johor             Johor Bahru   1723700.0
128478              Johor                  Kluang    323500.0
128877              Johor             Kota Tinggi    224900.0
129276              Johor                   Kulai    329200.0
129675              Johor                 Mersing     79100.0
130074              Johor                    Muar    314000.0
130473              Johor                 Pontian    173600.0
130872              Johor                 Segamat    198200.0
131271              Johor                 Tangkak    164600.0
131670              Kedah                  Baling    144900.0
132069              Kedah           Bandar Baharu     44700.0
132468              Kedah              Kota Setar    376800.0
132867              Kedah              Kuala Muda    550500.0
133266              Kedah             Kubang Pasu    249700.0
133665  

In [4]:
#Preprocess the hospital data
hospital_counts = hospital_df.groupby(['State', 'City']).size().reset_index(name='num_hospitals')
hospital_counts.rename(columns={'City': 'district', 'State': 'state'}, inplace=True)
print(str(hospital_counts))

                 state           district  num_hospitals
0                Johor         Batu Pahat              1
1                Johor        Johor Bahru              3
2                Johor             Kluang              1
3                Johor        Kota Tinggi              1
4                Johor          Kulaijaya              1
..                 ...                ...            ...
127         Terengganu   Kuala Terengganu              1
128         Terengganu              Setiu              1
129  W.P. Kuala Lumpur  W.P. Kuala Lumpur              4
130        W.P. Labuan        W.P. Labuan              1
131     W.P. Putrajaya     W.P. Putrajaya              2

[132 rows x 3 columns]


In [5]:
#Preprocess the clinic data
clinic_counts = clinic_df.groupby(['State', 'District']).size().reset_index(name='num_clinics')
clinic_counts.rename(columns={'State': 'state', 'District': 'district'}, inplace=True)

print(str(clinic_counts))

                 state            district  num_clinics
0                Johor   81900 Kota Tinggi            1
1                Johor          Batu Pahat           62
2                Johor  Jalan Hang Lekir 1            1
3                Johor         Johor Bahru           36
4                Johor              Kluang           29
..                 ...                 ...          ...
181         Terengganu              Marang           20
182         Terengganu               Setiu           17
183  W.P. Kuala Lumpur   W.P. Kuala Lumpur           24
184        W.P. Labuan         W.P. Labuan           12
185     W.P. Putrajaya      W.P. Putrajaya            4

[186 rows x 3 columns]


In [14]:
# Preprocess hospital bed data
beds_df['date'] = pd.to_datetime(beds_df['date'], dayfirst=True)
latest_bed_year = beds_df['date'].dt.year.max()
beds_filtered = beds_df[beds_df['date'].dt.year == 2022]

# Only district-specific entries, remove 'All'
beds_filtered = beds_df[
    (beds_df['date'].dt.year == 2022) &
    (beds_df['type'].str.lower() == 'all') &
    (~beds_df['district'].str.lower().str.contains('all', na=False))
]
bed_summary = beds_filtered.groupby(['state', 'district'])['beds'].sum().reset_index()
bed_summary.rename(columns={'beds': 'hospital_beds'}, inplace=True)

print(bed_summary.to_string() , bed_summary.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   state          153 non-null    object
 1   district       153 non-null    object
 2   hospital_beds  153 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 3.7+ KB
                 state                            district  hospital_beds
0                Johor                          Batu Pahat            610
1                Johor                         Johor Bahru           3042
2                Johor                              Kluang            382
3                Johor                         Kota Tinggi            158
4                Johor                           Kulaijaya             93
5                Johor                              Ledang             69
6                Johor                             Mersing             70
7                Johor                 

In [16]:
# Start with population (since it determines which districts exist)
merged = pop_summary.copy()

# Merge hospital beds
merged = merged.merge(bed_summary, on=['state', 'district'], how='left')

# Merge hospitals
merged = merged.merge(hospital_counts, on=['state', 'district'], how='left')

# Merge clinics
merged = merged.merge(clinic_counts, on=['state', 'district'], how='left')

In [17]:
merged.to_csv("district_health_summary.csv", index=False)
print(merged.head())

   state     district  population  hospital_beds  num_hospitals  num_clinics
0  Johor   Batu Pahat    497500.0          610.0            1.0         62.0
1  Johor  Johor Bahru   1723700.0         3042.0            3.0         36.0
2  Johor       Kluang    323500.0          382.0            1.0         29.0
3  Johor  Kota Tinggi    224900.0          158.0            1.0         42.0
4  Johor        Kulai    329200.0            NaN            NaN         14.0
